In [ ]:
#!pip install azure-ml-core

### Creating environments

In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-244518
Azure region: eastus2
Subscription id: 2c48c51c-bd47-40d4-abbe-fb8eabd19c8c
Resource group: aml-quickstarts-244518


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException


# Create compute cluster
# https://learn.microsoft.com/en-us/azure/machine-learning/how-to-train-scikit-learn?view=azureml-api-2/n# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.
cpu_cluster_name = "computePR1cluster"

try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, using that.')
except  ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)
    
compute_target.wait_for_completion(show_output=True)
print(compute_target.get_status().serialize())
    ###  ###
    

InProgress..
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2023-11-28T10:38:46.139000+00:00', 'errors': None, 'creationTime': '2023-11-28T10:38:38.037561+00:00', 'modifiedTime': '2023-11-28T10:38:47.784505+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT1800S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_D2_V2'}


### Logistic Regression for classification train.py Section

In [5]:
from azureml.widgets import RunDetails
#from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler, regularisation nad max iterations
psr  = RandomParameterSampling(
    {
        '--C' : choice(0.1,1,10,20,50,100,200,500),
        '--max_iter': choice(30,50,100,200,300)
    }
)

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1, slack_amount=None, delay_evaluation=0)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(source_directory = './', 
                      script = 'train.py', 
                      compute_target=compute_target,
                      environment= sklearn_env)


# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(hyperparameter_sampling=psr, 
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     policy=policy,
                                     run_config=src,
                                     max_total_runs=16,
                                     max_concurrent_runs=4)



In [6]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
hyperdrive_run = exp.submit(hyperdrive_config)

RunDetails(hyperdrive_run).show()
###  ###


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [8]:
hyperdrive_run

Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,HD_927b5962-849a-423a-a969-2bffd3ebc533,hyperdrive,Running,Link to Azure Machine Learning studio,Link to Documentation


In [48]:
(hyperdrive_run.get_status())

'Completed'

In [49]:
import joblib
# Get your best run and save the model from that run.
best_model = hyperdrive_run.get_best_run_by_primary_metric()

model = best_model.register_model(model_name='logreg', model_path='outputs/logreg_model.joblib')
print("Best model metrics:", best_model.get_metrics() )

#print("Best model details :",best_model.get_details())

# Returns a list of the files that are stored in association with the run.
#print("Best file names :",best_model.get_file_names())

### end of train.py,  next AutoMl way here ###

Best model metrics: {'Max iterations:': 50, 'Regularization Strength:': 0.1, 'Accuracy': 0.9115320949052786}


### AutoML Section

In [50]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
dataset = TabularDatasetFactory.from_delimited_files(['https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'])



In [51]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(dataset)

In [52]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.

#https://learn.microsoft.com/en-us/azure/machine-learning/how-to-configure-cross-validation-data-splits?view=azureml-api-1#default-data-splits-and-cross-validation-in-machine-learning
#logistic regression, binary classification

automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=dataset,
    label_column_name='y',
    n_cross_validations= 2)

In [53]:
# Submit your automl run
rrun = exp.submit(automl_config, show_output = False)
rrun.wait_for_completion()

####

2023-11-28:11:07:41,230 WARNING  [local_experiment_driver.py:142] Running on local machine. Note that local runs always run synchronously even if you use the parameter 'show_output=False'


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_502316f9-fbf5-4aab-aa47-ced3e54d038a,automl,Preparing,Link to Azure Machine Learning studio,Link to Documentation


2023/11/28 11:10:26 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!
2023-11-28:11:39:05,378 INFO     [explanation_client.py:334] Using default datastore for uploads


{'runId': 'AutoML_502316f9-fbf5-4aab-aa47-ced3e54d038a',
 'target': 'local',
 'status': 'Completed',
 'startTimeUtc': '2023-11-28T11:07:46.669711Z',
 'endTimeUtc': '2023-11-28T11:37:18.341996Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'No scores improved over last 10 iterations, so experiment stopped early. This early stopping behavior can be disabled by setting enable_early_stopping = False in AutoMLConfig for notebook/python SDK runs.'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '2',
  'target': 'local',
  'AMLSettingsJsonString': '{"path":null,"name":"udacity-project","subscription_id":"2c48c51c-bd47-40d4-abbe-fb8eabd19c8c","resource_group":"aml-quickstarts-244518","workspace_name":"quick-starts-ws-244518","region":"eastus2","compute_target":"local","spark_service":null,"azur

In [54]:
# Retrieve and save your best automl model.
best_run, obtained_model =rrun.get_output()


# Returns the metrics
print("Best run metrics :")

best_run.get_metrics()

#print("Best run details :",best_run.get_details())

#print("Best file names :",best_run.get_file_names())

#obtained_model

### ###


Best run metrics :


{'precision_score_macro': 0.7946540419838736,
 'AUC_weighted': 0.9468508365821191,
 'matthews_correlation': 0.5657969295740102,
 'log_loss': 0.26599350192122495,
 'recall_score_micro': 0.9167830045523521,
 'precision_score_weighted': 0.9137486097039135,
 'norm_macro_recall': 0.5432257892007074,
 'average_precision_score_weighted': 0.9546857060896136,
 'weighted_accuracy': 0.9528466334064793,
 'recall_score_weighted': 0.9167830045523521,
 'AUC_micro': 0.9803419113431167,
 'f1_score_macro': 0.7824628739546196,
 'precision_score_micro': 0.9167830045523521,
 'average_precision_score_macro': 0.821903197962426,
 'f1_score_micro': 0.9167830045523521,
 'recall_score_macro': 0.7716128946003538,
 'f1_score_weighted': 0.9150928041182007,
 'AUC_macro': 0.9468508365821192,
 'accuracy': 0.9167830045523521,
 'average_precision_score_micro': 0.9811134158535803,
 'balanced_accuracy': 0.7716128946003538,
 'confusion_matrix': 'aml://artifactId/ExperimentRun/dcid.AutoML_502316f9-fbf5-4aab-aa47-ced3e54d038

In [55]:
estimator= obtained_model.steps[-1]
estimator

('prefittedsoftvotingclassifier',
 PreFittedSoftVotingClassifier(classification_labels=numpy.array([0, 1]), estimators=[('11', Pipeline(memory=None, steps=[('standardscalerwrapper', StandardScalerWrapper(copy=True, with_mean=False, with_std=False)), ('xgboostclassifier', XGBoostClassifier(booster='gbtree', colsample_bytree=0.6, eta=0.3, gamma=0, max_depth=6, max_leaves=0, n_estimators=10, n_jobs=1, objective='reg:logistic', problem_info=ProblemInfo(gpu_training_param_dict={'processing_unit_type': 'cpu'}), random_state=0, reg_alpha=0.3125, reg_lambda=2.3958333333333335, subsample=1, tree_method='auto'))], verbose=False)), ('25', Pipeline(memory=None, steps=[('standardscalerwrapper', StandardScalerWrapper(copy=True, with_mean=False, with_std=False)), ('xgboostclassifier', XGBoostClassifier(booster='gbtree', colsample_bytree=1, eta=0.05, gamma=0, max_depth=6, max_leaves=0, n_estimators=200, n_jobs=1, objective='reg:logistic', problem_info=ProblemInfo(gpu_training_param_dict={'processing_u

In [56]:
obtained_model[0]

DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, task='classification')

In [57]:
# Delete the recourses, the cluster
compute_target.delete()

Current provisioning state of AmlCompute is "Deleting"

Current provisioning state of AmlCompute is "Deleting"

